## Breadth-First Search

In [1]:
# Import numpy, Enum and Queue
import numpy as np
from enum import Enum
from queue import Queue

In [2]:
# Define a start and goal location
start = (0, 0)
goal = (4, 4)

# Define your grid-based state space of obstacles and free space
grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

 [S, 1, 0, 0, 0, 0] 
 
 [0, 1, 0, 0, 0, 0]
 
 [0, 1, 0, 1, 0, 0]
 
 [0, 0, 0, 1, 1, 0]
 
 [0, 0, 0, 1, E, 0]

In [3]:
# Define your action set using Enum()
class Action(Enum): 
    # Actions are tuples corresponding to movements in (i, j)
    LEFT = (0, -1)
    RIGHT = (0, 1)
    UP = (-1, 0)
    DOWN = (1, 0)
    
    # Define string characters for each action
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'

In [4]:
# Define a function that returns a list of valid actions 
# through the grid from the current node
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    # First define a list of all possible actions
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN]
    # Retrieve the grid shape and position of the current node
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    # check if the node is off the grid or it's an obstacle
    # If it is either, remove the action that takes you there
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
        
    return valid

In [5]:
# Define a function to visualize the path
def visualize_path(grid, path, start):
    """
    Given a grid, path and start position
    return visual of the path to the goal.
    
    'S' -> start 
    'G' -> goal
    'O' -> obstacle
    ' ' -> empty
    """
    # Define a grid of string characters for visualization
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    # Fill in the string grid
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    
    return sgrid

In [6]:
# Null path
visualize_path(grid, [], start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       [' ', 'O', ' ', ' ', ' ', ' '],
       [' ', 'O', ' ', 'O', ' ', ' '],
       [' ', ' ', ' ', 'O', 'O', ' '],
       [' ', ' ', ' ', 'O', ' ', ' ']], 
      dtype='<U1')

### Breadth-First Algorithm

In [13]:
# Define your breadth-first search function here
def breadth_first(grid, start, goal):

    queue = Queue()
    # start by putting the start sate into the queue
    queue.put(start)

    visited = set()
    visited.add(start)
    
    branch = {}
    found = False
    
    # Run loop while queue is not empty
    while not queue.empty():
        # Remove the first element from the queue
        current_node = queue.get()
        
        # Check if the current node corresponds to the goal state
        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            # TODO: Get the new nodes connected to the current node
            # Iterate through each of the new nodes and:
            # If the node has not been visited you will need to
            # 1. Mark it as visited
            # 2. Add it to the queue
            # 3. Add how you got there to the branch dictionary 
            for action in valid_actions(grid, current_node):
                # delta of performing the action
                partial_action = action.value
                next_node = (current_node[0] + partial_action[0], current_node[1] + partial_action[1])
                if next_node not in visited:                
                    visited.add(next_node)               
                    queue.put(next_node)
                    branch[next_node] = (current_node, action)
    
    # Now, if you found a path, retrace your steps through 
    # the branch dictionary to find out how you got there!
    
    print(branch)
    
    path = []
    if found:
        # retrace steps
        path = []
        n = goal
        while branch[n][0] != start:
            path.append(branch[n][1])
            n = branch[n][0]
        path.append(branch[n][1])
            
    return path[::-1]

### Executing the search

Run `breadth_first()` and reference the grid to see if the path makes sense.

In [14]:
path = breadth_first(grid, start, goal)
print(path)

Found a path.
{(3, 2): ((3, 1), <Action.RIGHT: (0, 1)>), (1, 3): ((1, 2), <Action.RIGHT: (0, 1)>), (3, 0): ((2, 0), <Action.DOWN: (1, 0)>), (0, 4): ((0, 3), <Action.RIGHT: (0, 1)>), (1, 4): ((1, 3), <Action.RIGHT: (0, 1)>), (0, 5): ((0, 4), <Action.RIGHT: (0, 1)>), (1, 0): ((0, 0), <Action.DOWN: (1, 0)>), (4, 2): ((3, 2), <Action.DOWN: (1, 0)>), (0, 3): ((0, 2), <Action.RIGHT: (0, 1)>), (2, 5): ((1, 5), <Action.DOWN: (1, 0)>), (3, 5): ((2, 5), <Action.DOWN: (1, 0)>), (1, 2): ((2, 2), <Action.UP: (-1, 0)>), (2, 0): ((1, 0), <Action.DOWN: (1, 0)>), (3, 1): ((3, 0), <Action.RIGHT: (0, 1)>), (4, 4): ((4, 5), <Action.LEFT: (0, -1)>), (0, 2): ((1, 2), <Action.UP: (-1, 0)>), (1, 5): ((1, 4), <Action.RIGHT: (0, 1)>), (2, 2): ((3, 2), <Action.UP: (-1, 0)>), (4, 5): ((3, 5), <Action.DOWN: (1, 0)>), (4, 0): ((3, 0), <Action.DOWN: (1, 0)>), (4, 1): ((3, 1), <Action.DOWN: (1, 0)>), (2, 4): ((1, 4), <Action.DOWN: (1, 0)>)}
[<Action.DOWN: (1, 0)>, <Action.DOWN: (1, 0)>, <Action.DOWN: (1, 0)>, <Action

In [12]:
# S -> start, G -> goal, O -> obstacle
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       ['v', 'O', '>', '>', '>', 'v'],
       ['v', 'O', '^', 'O', ' ', 'v'],
       ['>', '>', '^', 'O', 'O', 'v'],
       [' ', ' ', ' ', 'O', 'G', '<']], 
      dtype='<U1')